# Practical session 4 - K-nearest neighbours (K-NN) classification with numpy, scikit-learn, cython and numba

Students (pair):
- [Jérémy Jean](https://github.com/heez77)
- [Maxime Gey](https://github.com/Purjack)

**Useful references for this lab**:

[1] scikit-learn: [documentation](https://scikit-learn.org/stable/modules/neighbors.html?highlight=knn%20classification)

[2] `numba`: [documentation](http://numba.pydata.org/) 

[3] cython: [a very useful tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial), and [another one](http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html)



## <a name="content">Contents</a>
- [Exercise 1: KNN classification with numpy and sklearn](#ex1)
- [Exercise 2: Code acceleration with cython](#ex2)
- [Exercise 3: Code acceleration with numba](#ex3)
---

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <a name="ex1">Exercise 1: K-Nearest Neighbours (K-NN) classification with numpy and scikit-learn</a> [(&#8593;)](#content)

This session is a first introduction to classification using the most intuitive non parametric method: the $K$-nearest neighbours. The principle is [the following](https://scikit-learn.org/stable/modules/neighbors.html?highlight=knn%20classification). A set of labelled observations is given as a learning set. A classification taks then consists in assigning a label to any new observation. In particular, the K-NN approach consists in assigning to the observation the most frequent label among its $K$ nearest neighbours taken in the training set.

### A. Validation on synthetic data

Load the training and test datasets `data/synth_train.txt` and `data/synth_test.txt`. Targets belong to the set $\{1,2\}$ and entries belong to $\mathbb{R}^2$. The file `data/synth_train.txt` contain 100 training data samples, and `data/synth_test.txt` contains 200 test samples, where:

- the 1st column contains the label of the class the sample;
- columns 2 & 3 contain the coordinates of each sample (in $\mathbb{R}^2$).

Useful commands can be found below.

```python
# load the training set
train = np.loadtxt('data/synth_train.txt')  #...,delimiter=',') if there are ',' as delimiters
class_train = train[:,0]
x_train = train[:,1:]
N_train = train.shape[0]
```

```python
# load the test set
test = np.loadtxt('/datasynth_test.txt') 
class_test_1 = test[test[:,0]==1]
class_test_2 = test[test[:,0]==2]
x_test = test[:,1:]
N_test = test.shape[0]
```

1\. Display the training set and distinguish the two classes. 

> Hint: useful functions include `matplotlib.pyplot.scatter` or `matplotlib.pyplot.plot`.

**Answer:**

In [19]:
import plotly.express as px
import numpy as np
import pandas as pd

train = np.loadtxt('data/synth_train.txt')  
df_train = pd.DataFrame(train, columns = ['classe', 'x1', 'x2'])
N_train = df_train.size

test = np.loadtxt('data/synth_test.txt') 
df_test = pd.DataFrame(test, columns = ['classe', 'x1', 'x2'])
N_test = df_test.size

In [20]:
px.scatter(df_train,x='x1',y='x2',color='classe',title='Data visualization of the training dataset')

In [21]:
px.scatter(df_test,x='x1',y='x2',color='classe',title='Data visualization of the test dataset')

2. Implement the K-nearest neighbours algorithm for classification.

Hint:

- useful functions include numpy.linalg.norm, numpy.argsort, numpy.bincount;
- implement the algorithm as a function rather than an object. This will drastically simplify the acceleration step using Cython.
- for an optimized partial sorting procedure, you may have a look at the bottleneck.argpartition function.

**Answer:**

In [22]:
class K_nearest_neighbors():
    """K nearest neighbors implementation.
    """
    def __init__(self, train:np.ndarray, test:np.ndarray, K:int):
        """
        Args:
            train (np.ndarray): The training dataset
            test (np.ndarray): The test dataset
            K (int): K value in the nearest neighbors implementation, must be strictly positive.
        """
        self.x_train = train[:,1:]
        self.y_train = train[:,0]
        self.N_train = train.shape[0]
        self.x_test = test[:,1:]
        self.y_test = test[:,0]
        self.N_test = test.shape[0]
        assert K>0, "K must be strictly positive"
        self.K = K

    def classifier(self, x_new:np.ndarray)->float:
        """Compute the classification using K nearest neighbors algorithm method.

        Args:
            x_new (np.ndarray): The input to be predicted.

        Returns:
            float: 1. if x_new is in class 1 else 2.
        """
        distances = []
        counter1 = 0
        counter2 = 0
        for i in range(self.N_train):
            distances.append((np.linalg.norm(x_new-self.x_train[i]),self.y_train[i]))
        dtype = [('distance', float), ('target', float)]
        distances=np.array(distances,dtype=dtype)
        distances=np.sort(distances,order='distance')  # plus petites en premier
        for j in range(0,self.K):
            if distances[j][1]==1.:
                counter1+=1
            if distances[j][1]==2.:
                counter2+=1
            if counter1+counter2==self.K:
                break
        if counter1>counter2:
            return 1.
        else:
            return 2.

    def predict(self)->np.ndarray:
        """Compute the prediction for the test dataset.

        Returns:
            np.ndarray: An array in dimension 1 with all the predictions.
        """
        predictions = np.zeros(self.N_test, dtype=float)
        for i in range(self.N_test):
            predictions[i] = self.classifier(self.x_test[i])
        return predictions

    def error_rate(self)->float:
        """Compute the error rate for the test dataset.

        Returns:
            float: Error rate value, a float between 0 and 1.
        """
        predictions = self.predict()
        error_rate = 1-np.count_nonzero(predictions==self.y_test)/self.N_test
        return error_rate
        

3\. Compute the error rate on the training set and the test set for $K \in \{1,2, \dotsc, 20\}$. Display the classification result (see 1.) for the configuration with the lowest error rate.

**Answer:**

In [23]:
def error_rate_analyse(min:int,max:int, train:np.ndarray, test:np.ndarray)->pd.DataFrame:
    """Make an analysis of the error rate for a train and test dataset with different values of K. Compute the error rate for K values between min and max with a step of 1.

    Args:
        min (int): The min value for K in the K nearest neighbors algorithm.
        max (int): The max value for K in the K nearest neighbors algorithm.
        train (np.ndarray): Training dataset.
        test (np.ndarray): Test dataset.

    Returns:
        pd.DataFrame: Returns a DataFrame with two columns, one for the K value and a second for the error rate.
    """
    K_values = np.arange(min,max+1)
    error_rate = np.zeros(max-min+1,dtype=float)
    for i,K in enumerate(K_values):
        Knn_classifier = K_nearest_neighbors(train, test, K)
        error_rate[i] = Knn_classifier.error_rate()
    return pd.DataFrame(dict(K=K_values,error_rate=error_rate))
    

In [24]:
df_analyse_error_rate = error_rate_analyse(1,20,train, test)

In [25]:
px.line(df_analyse_error_rate,x='K', y='error_rate', title='Evolution of the error rate')

The best K value is 3.

We obtain this classification :

In [26]:
Knn_classifier = K_nearest_neighbors(train, test, 3)
predictions = Knn_classifier.predict()
df_predictions = df_test.copy()
df_predictions['predictions'] = predictions
px.scatter(df_predictions ,x='x1',y='x2',color='predictions', title='Visualization of the classification with K=3')

5\. Compare the results of you implementation with those of [`sklearn.neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html?highlight=kneighborsclassifier#sklearn.neighbors.KNeighborsClassifier). Compare the runtime of these two versions using the [`timeit`](https://docs.python.org/3/library/timeit.html) module (see session 1).

**Answer:**

In [27]:
from sklearn.neighbors import KNeighborsClassifier

Knn_sklearn = KNeighborsClassifier(3)
Knn_sklearn.fit(train[:,1:], train[:,0])
sklearn_predictions = Knn_sklearn.predict(test[:,1:])
error_rate_sklearn = 1-np.count_nonzero(predictions==test[:,0])/test.shape[0]
print(error_rate_sklearn)

0.04500000000000004


We have the same error rate which is logical because the algorithm is deterministic. 

In [28]:
import timeit

loop = 10
execution_time = timeit.timeit(lambda: Knn_classifier.predict(), number=loop) / loop

execution_time_sklearn = timeit.timeit(lambda: Knn_sklearn.fit(train[:,1:], train[:,0]), number=loop) / loop + timeit.timeit(lambda: Knn_sklearn.predict(test[:,1:]), number=loop) / loop

print(execution_time/execution_time_sklearn)

29.951262649847006


The execution time of the implementation by sklearn is much faster than ours (factor 20).

### B. Application to a real dataset (Breast cancer Wisconsin).

6\. Apply the K-NN classifier to the real dataset `data/wdbc12.data.txt.` Further details about the data are provided in `data/wdbc12.names.txt`.

> Hint: you can use the function [`train_test_split` from `sklearn.model_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the dataset into a training and a test set.

**Answer:**

In [29]:
from sklearn.model_selection import train_test_split

data = np.loadtxt('data/wdbc12.data.txt', delimiter=',')[:,1:]
train_cancer, test_cancer = train_test_split(data, random_state=420)

In [30]:
df_analyse_error_rate = error_rate_analyse(1,20,train_cancer, test_cancer)

In [31]:
px.line(df_analyse_error_rate,x='K', y='error_rate', title='Evolution of the error rate')

The best value of K is 11. We obtain an error rate of value 0.049.

## <a name="ex2">Exercise 2: Code acceleration with cython</a> [(&#8593;)](#content)

Cython allows C code to be easily interfaced with Python. It can be useful to make your code faster for a small coding effort, in particular when using loops. A general approach to optimize your code is outlined in the [Scipy lecture notes, Section 2.4](https://scipy-lectures.org/advanced/optimizing/index.html). Complementary reading about interfacing Python with C can be found in [Section 2.8](https://scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html).

1\. Read carefully the [cython tutorial](http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html), which describes step by the step how the toy example reported below has been developed.

**Setup**: Compile the toy example provided in `example_cy/` by running, in the command line (anaconda prompt on windows)

```bash
cd example_cy && python setup.py build_ext --inplace
```

Note that the compilation process has been slightly automatised with the instructions reported in `example_cy/setup.py`. To test the module, run

In [35]:
!cd example_cy && python setup.py build_ext --inplace

running build_ext
building 'example_cy.helloworld' extension
creating build
creating build\temp.win-amd64-3.10
creating build\temp.win-amd64-3.10\Release
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\include -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\um" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\shared" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\winrt" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\cppwinrt" /Tchelloworld.c 

error: could not create 'example_cy\helloworld.cp310-win_amd64.pyd': No such file or directory


In [37]:
import example_cy.helloworld as toy

toy.printhello()

Hello World


which should display
```python
Hello World
```

> Warning: 
> - do not forget to include an empty `__init__.py` file in the directory where your source code lives (`import` will fail if this is not the case).
> - in case you have any setup issue, take a look at the `notes.md` file.
> - if the C code and/or the executable do not seem to be regenerated by the build instructions, delete the C code and the executable first, and re-execute the compilation afterwards.
> - do not hesitate to restart the Python kernel if necessary when the Cython executable has been re-generated.

2\. Read the [Numpy/Cython tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial), focussing on the paragraphs **Cython at a glance**, and **Your Cython environment** until **"More generic code"**. An example to compile a `.pyx` file depending on `numpy` is included in `example_np_cy/`.

> Remarks: 
> - the `annotate=True` flag in the `setup.py` allows an additional `.html` document to be generated (`<your_module_name>.html`), showing, for each line of the Cython code, the associated C instructions generated. Highlighted in yellow are the interactions with Python: the darker a region appears, the less efficient the generated C code is for this section. Work in priority on these! 
> - make sure all the previously generated files are deleted to allow the .html report to be generated;
> - if you are working on your own machine and don't have a C/C++ compiler installed, read the notes provided in `notes.md`;
> - use `cdef` for pure C functions (not exported to Python), `cpdef` should be favored for functions containing C instructions and later called from Python.

**Answer:**

In [49]:
!cd example_np_cy && python setup.py build_ext --inplace

array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)

a = 4
b = 3
c = 9

import compute_cy as cpt
cpt.compute(array_1, array_2, a, b, c)

Compiling compute_cy.pyx because it changed.
[1/1] Cythonizing compute_cy.pyx
running build_ext
building 'compute_cy' extension
creating build
creating build\temp.win-amd64-3.10
creating build\temp.win-amd64-3.10\Release
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\include -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\um" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\shared" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\winrt" "-IC:\Program Files (x86

array([[2893,  514, 1906, ..., 1741, 2797, 2353],
       [2083,  679,  535, ..., 2083,  649, 1849],
       [2080, 1027, 3019, ...,  973,  412, 3031],
       ...,
       [ 250, 1654, 2218, ...,  760, 2242, 1198],
       [ 799,   52, 1018, ..., 1486, 1780,   67],
       [2842, 2389, 1417, ...,  214, 2863, 1981]], dtype=int32)

In [55]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [63]:
def execution_without_cython():
    from example_np_cy.compute import clip, compute
    import numpy as np
    array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
    array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)

    a = 4
    b = 3
    c = 9
    compute(array_1, array_2, a, b, c)

In [64]:
def execution_with_cython():
    %%cython
    from example_np_cy.compute import clip, compute
    import numpy as np
    
    array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
    array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)

    a = 4
    b = 3
    c = 9
    compute(array_1, array_2, a, b, c)

In [65]:
import timeit

loop = 10
execution_time = timeit.timeit(lambda: execution_without_cython, number=loop) / loop

execution_time_cython = timeit.timeit(lambda: execution_with_cython, number=loop) / loop

print(execution_time/execution_time_cython)

1.2727272727272727


The <i>compute</i> function is already computed using numpy and therfore really fast. It seems overkill to use Cython in this case buut we still notice a 1.27 factor in terms of rapidity.

3\. Use Cython to implement a faster version of the numpy K-NN classifier implemented in [Exercise 1](#ex1). To do so, apply step-by-step the techniques introduced in the [Numpy/Cython tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial) (*i.e.*, compile and time your code after each step to report the evolution, keeping track of the different versions of the cython function).

> Hint: if you keep numpy arrays, make sure you use memory views (see numpy/cython tutorial) to access the elements within it. Be extremely careful with the type of the input arrays (you may need to recast the format of the input elements before entering the function. The `numpy.asarray` function can prove useful).

> **Detailed guidelines**: a few notes and *caveat* to help you re-writing your code in cython:
> - try to reduce the number of calls to numpy instructions as much as possible;
> - **you do not have to optimize everything**. For the KNN function above, most of the time is spent in computing euclidean distances: you can thus focus on optimizing tihs operations by explicitly writing a for loop, which will ensure a minimal interaction with numpy when generating the associated C code at compilation. Calls to other numpy functions can be kept as-is;
> - if you need to create an array within the cython function, used np.zeros (**do NOT use python lists**), and use a memory view to access its content;
> - specify the type for all variables and numpy arrays. Pay attention to the type of the input arrays passed to the Cython function;
> - whenever an array is returned, use memory views and index(es) to efficiently access its content;
> - some numpy operators (e.g., broadcasting mechanism) do not work with memory views. In this case, you can directly write for loop(s) to encode the operation of interest (the loops will be optimized out at compile time);
> - only use at the final development stage the following cython optimization (not before, as they can crash the program without any help):
>
>```python
>@cython.boundscheck(False)
>@cython.wraparound(False)
>```

**Answer:**

In [146]:
!cd knn && python setup.py build_ext --inplace

Compiling cython_metric.pyx because it changed.
[1/1] Cythonizing cython_metric.pyx
running build_ext
building 'cython_metric' extension
creating build
creating build\temp.win-amd64-3.10
creating build\temp.win-amd64-3.10\Release
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\include -Ic:\Users\geyma\AppData\Local\Programs\Python\Python310\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.33.31629\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\um" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\shared" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0.19041.0\\winrt" "-IC:\Program F

In [147]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from knn.cython_metric import mydist
import time

In [151]:
def dist(x, y):
  return np.sum((y-x)**2)

In [155]:
def execution_cython():
    classifier_cython = KNeighborsClassifier(3, metric=mydist)

    t1 = time.time()
    classifier_cython.fit(train[:,1:], train[:,0])

    res = classifier_cython.predict(test[:,1:])
    t2 = time.time()

In [156]:
def execution():
    classifier = KNeighborsClassifier(3, metric=dist)

    t1 = time.time()
    classifier.fit(train[:,1:], train[:,0])

    res = classifier.predict(test[:,1:])
    t2 = time.time()

4\. Compare the runtime of the two algorithms (using `timeit.timeit`), and conclude about the interest of using cython in this case.

**Answer:**

In [157]:
import timeit

loop = 10
execution_time = timeit.timeit(lambda: execution_cython, number=loop) / loop

execution_time_cython = timeit.timeit(lambda: execution, number=loop) / loop

print(execution_time/execution_time_cython)

2.090909090909091


Even with a KNN very optimized by the sklearn library, we see that Cython allows us to compute two times faster !

## <a name="ex3">Exercise 3: Code acceleration with numba</a> [(&#8593;)](#content)

`numba` is a just-in-time (JIT) compiler which translates Python codes into efficient machine code at runtime. A significant acceleration can be obtained by adding a few simple decorators to a standard Python function, up to a few restrictions detailed [here](http://numba.pydata.org/numba-doc/latest/user/performance-tips.html).

If you have written most of the KNN classifier of exercise 1 with numpy, there is little to no chance that you will get an acceleration with numba (justifying the use of cython in this case). An interesting acceleration factor can however be obtained for the computation of the total variation investigated in session 2.

1\. Take a look at the [numba 5 min tour](http://numba.pydata.org/numba-doc/latest/user/5minguide.html), and accelerate the total variation code from session 2 with the `@jit` decorator. You may have to rewrite small portions of your code to get the expected acceleration (see [performance tips](http://numba.pydata.org/numba-doc/latest/user/performance-tips.html)).

**Answer:**

In [ ]:
from numba import njit

Comparison for Knn : 

In [ ]:
import numba


### We put the predict and classifier functions in one function
@njit
def predict_K_nearest_neighbors_with_numba(x_train:np.ndarray,y_train:np.ndarray, x_test:np.ndarray, K:int):
    N_test = test.shape[0]
    N_train = x_train.shape[0]
    predictions = np.zeros(N_test, dtype=float)
    for i in range(N_test):   
        distances = []
        targets = []
        counter1 = 0
        counter2 = 0
        for j in range(N_train):
            # separation in two different list to help numba
            distances.append(np.linalg.norm(x_test[i]-x_train[j]))
            targets.append(y_train[j])
        distances=np.array(distances,dtype=np.float64)
        idx_sort = np.argsort(distances)
        for k in range(0,K):
            if targets[idx_sort[k]]==1.:
                counter1+=1
            if targets[idx_sort[k]]==2.:
                counter2+=1
            if counter1+counter2==K:
                break
        if counter1>counter2:
            predictions[i] = 1.
        else:
            predictions[i] = 2.
    return predictions


def predict_K_nearest_neighbors(x_train:np.ndarray,y_train:np.ndarray, x_test:np.ndarray, K:int):
    N_test = test.shape[0]
    N_train = x_train.shape[0]
    predictions = np.zeros(N_test, dtype=float)
    for i in range(N_test):   
        distances = []
        targets = []
        counter1 = 0
        counter2 = 0
        for j in range(N_train):
            # separation in two different list to help numba
            distances.append(np.linalg.norm(x_test[i]-x_train[j]))
            targets.append(y_train[j])
        distances=np.array(distances,dtype=np.float64)
        idx_sort = np.argsort(distances)
        for k in range(0,K):
            if targets[idx_sort[k]]==1.:
                counter1+=1
            if targets[idx_sort[k]]==2.:
                counter2+=1
            if counter1+counter2==K:
                break
        if counter1>counter2:
            predictions[i] = 1.
        else:
            predictions[i] = 2.
    return predictions

In [ ]:
_ = predict_K_nearest_neighbors_with_numba(train[:,1:], train[:,0], test[:,1:], 3)

In [ ]:
loop = 10
execution_time = timeit.timeit(lambda: predict_K_nearest_neighbors(train[:,1:], train[:,0], test[:,1:], 3), number=loop) / loop

execution_time_numba = timeit.timeit(lambda: predict_K_nearest_neighbors_with_numba(train[:,1:], train[:,0], test[:,1:], 3), number=loop) / loop

print(execution_time/execution_time_numba)

This implementation is 10 time faster with numba.

Comparison for Total variation :

In [ ]:
from numba import jit

def gradient2D(X:np.array)->tuple:
    """This function computes the 2D discrete gradient operator D applied to a matrix X of dimensions 2

    Args:
        X (array): A matrix in C^(M,N)

    Returns:
        (array,array) : A tuple in C^(M,N) x C^(M,N)
    """
    assert X.ndim <=2, "The input array has more than 2 dimensions"

    XDh = np.zeros(X.shape)
    for n in range(1,XDh.shape[1]):
        XDh[:,n-1] = X[:,n]-X[:,n-1]

    DvX = np.zeros(X.shape)
    for m in range(1,XDh.shape[0]):
        DvX[m-1,:] = X[m,:]-X[m-1,:]


    return XDh, DvX

def tv(X:np.array)->float:
    """This function compute the discrete isotropic total variation of an input matrix in C^(M,N)

    Args:
        X (np.array): A matrix in C^(MxN)

    Returns:
        float: returns the value of the TV for the input matrix X
    """
    XDh, DvX = gradient2D(X)
    sum = 0
    for m in range(XDh.shape[0]):
        for n in range(XDh.shape[1]):
            sum+= np.sqrt(XDh[m,n]**2 + DvX[m,n]**2)
    return sum


@njit
def gradient2D_numba(X:np.array)->tuple:
    """This function computes the 2D discrete gradient operator D applied to a matrix X of dimensions 2

    Args:
        X (array): A matrix in C^(M,N)

    Returns:
        (array,array) : A tuple in C^(M,N) x C^(M,N)
    """
    assert X.ndim <=2, "The input array has more than 2 dimensions"

    XDh = np.zeros(X.shape)
    for n in range(1,XDh.shape[1]):
        XDh[:,n-1] = X[:,n]-X[:,n-1]

    DvX = np.zeros(X.shape)
    for m in range(1,XDh.shape[0]):
        DvX[m-1,:] = X[m,:]-X[m-1,:]


    return XDh, DvX

@njit
def tv_numba(X:np.array)->float:
    """This function compute the discrete isotropic total variation of an input matrix in C^(M,N)

    Args:
        X (np.array): A matrix in C^(MxN)

    Returns:
        float: returns the value of the TV for the input matrix X
    """
    XDh, DvX = gradient2D_numba(X)
    sum = 0
    for m in range(XDh.shape[0]):
        for n in range(XDh.shape[1]):
            sum+= np.sqrt(XDh[m,n]**2 + DvX[m,n]**2)
    return sum

2\. Compare the runtime of the your numpy implementation and the `numba`-accelerated version (using `timeit.timeit`). 
> **Warning**: first run the numba version once to trigger the compilation, and then time it as usual. This is needed to avoid including the JIT compilation step in the runtime.

**Answer:**

In [ ]:
loop = 100
rng = np.random.default_rng(84548)
M,N = rng.integers(low=50, high=100, size = 2)
X = rng.random((M,N))
tv_numba(X)

In [ ]:
loop = 100
rng = np.random.default_rng(84548)
M,N = rng.integers(low=50, high=100, size = 2)
X = rng.random((M,N))

execution_time = timeit.timeit(lambda: tv(X), number=loop) / loop

execution_time_numba = timeit.timeit(lambda: tv_numba(X), number=loop) / loop

print(execution_time/execution_time_numba)

The implementation with numba is 230 time faster which is really better than for Knn classifier. For this implementation we really see the advantages of numba.